In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime

# 1. 데이터 로드 (전처리된 파일 혹은 원본 파일)
# 본인의 실제 파일 경로로 수정하세요.
file_path = '../database/reviews_imdb.csv' 
df = pd.read_csv(file_path)

# 2. 분포 파악을 위한 기초 변수 생성
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df["char_len"] = df["content"].str.len()
# 문장 수 계산 (파생 변수 예시)
df['sentence_count'] = df['content'].apply(lambda x: len(str(x).split('.')) if pd.notnull(x) else 0)

# 3. 저장 폴더 설정
PLOT_DIR = 'review_analysis/plots'
os.makedirs(PLOT_DIR, exist_ok=True)
sns.set_theme(style="whitegrid")

print("=== [EDA] 데이터 분포 및 이상치 파악 시작 ===")

# 1) Rating 분포 (별점)
plt.figure(figsize=(10, 4))
df["rating"].dropna().hist(bins=20)
plt.title("rating(IMDb)")
plt.xlabel("rating")
plt.ylabel("count")
plt.tight_layout()
plt.savefig(os.path.join(PLOT_DIR, "rating(IMDb).png"))
plt.close()

# 2) Review length 분포
plt.figure(figsize=(10, 4))
df["char_len"].hist(bins=30)
plt.title("review_length(IMDb)")
plt.xlabel("char_len")
plt.ylabel("count")
plt.tight_layout()
plt.savefig(os.path.join(PLOT_DIR, "review_length(IMDb).png"))
plt.close()

# 3) Daily review counts
df_ts = df[df["date"].notna()].copy()
df_ts["date_only"] = df_ts["date"].dt.date
daily_counts = df_ts["date_only"].value_counts().sort_index()

plt.figure(figsize=(14, 4))
plt.plot(daily_counts.index, daily_counts.values)
plt.title("review_counts(IMDb)")
plt.xlabel("date")
plt.ylabel("count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(PLOT_DIR, "review_counts(IMDb).png"))
plt.close()

# --- [4. 이상치 수치 파악 (터미널 출력)] ---
upper_limit = df['char_len'].quantile(0.99)

print(f"\n1. 별점 이상치 (1-10 범위 밖): {len(df[(df['rating'] < 1) | (df['rating'] > 10)])}개")
print(f"2. 길이 이상치 (10자 미만): {len(df[df['char_len'] < 10])}개")
print(f"3. 기간 이상치 (2014년 이전): {len(df[df['date'] < '2014-01-01'])}개")
print(f"4. 미래 날짜 데이터: {len(df[df['date'] > datetime.now()])}개")


=== [EDA] 데이터 분포 및 이상치 파악 시작 ===
1. 별점 이상치 (1-10 범위 밖): 35개
2. 길이 하한 이상치 (3자 미만): 0개
3. 길이 상한 이상치 (상위 1%, 5684.8자 초과): 6개
4. 기간 이상치 (2014년 이전): 0개
5. 미래 날짜 데이터: 0개
